# **Data Pre-Processing**

In [1]:
import pandas as pd

### **List of Traffic Data Collection Stations**

1. Read the .csv file and store in a DataFrame:

In [2]:
trafficStations = pd.read_csv('datasets/road_traffic_counts_station_reference.csv')

2. Drop unnecesary columns:

In [3]:
# drop 'the_geom', 'the_geom_webmercator', 'cartodb_id', 'record_id', 'station_id', 'road_number', 'link_number', 'road_name', 'common_road_name', 'secondary_name', 'road_name_base', 'road_name_type', 'mab_identifier', 'road_classification_admin', 'lambert_easting', 'lambert_northing', 'publish', 'md5', 'updated_on', 'device_type'
trafficStations = trafficStations.drop(columns=['the_geom','the_geom_webmercator','cartodb_id','record_id','station_id','road_number','link_number','road_name','common_road_name','secondary_name','road_name_base','road_name_type','mab_identifier','road_classification_admin','lambert_easting','lambert_northing','publish','md5', 'updated_on', 'device_type'])

3. Manually fill in the rms_region, lga, suburb, and post_code columns for the two entries with NULL values.  
*(The NULL values occurred because the stations are located on the boundary between two postcodes. I used their coordinates in Google Maps to determine an appropriate postcode for each.)*

In [4]:
# updating row at index 589
trafficStations.loc[589, ['rms_region', 'lga', 'suburb', 'post_code']] = ['Sydney', 'Sydney', 'Chippendale', 2008]

# updating row at index 1668
trafficStations.loc[1668, ['rms_region', 'lga', 'suburb', 'post_code']] = ['Sydney', 'The Hills', 'Baulkham Hills', 2153]

4. Store cleaned DataFrame into a new .csv file:

In [5]:
trafficStations.to_csv('datasets_cleaned/trafficStations.csv', index=False)

### **Historical Traffic Data**

1. Read the .csv files and store in a DataFrame:

In [6]:
# read each CSV file into its own DataFrame
df0 = pd.read_csv('datasets/road_traffic_counts_hourly_permanent0.csv')
df1 = pd.read_csv('datasets/road_traffic_counts_hourly_permanent1.csv', low_memory=False) # low_memory=False to avoid dtype warning
df2 = pd.read_csv('datasets/road_traffic_counts_hourly_permanent2.csv')
df3 = pd.read_csv('datasets/road_traffic_counts_hourly_permanent3.csv')

# concatenate the DataFrames into a single DataFrame
trafficData = pd.concat([df0, df1, df2, df3])

2. Extract the day from the date column:

In [7]:
# extract the day from the date column
trafficData['date'] = pd.to_datetime(trafficData['date'])
trafficData['day'] = trafficData['date'].dt.day

# get the list of columns
cols = list(trafficData.columns)

# find the index of the month column and insert the day column right after
month_idx = cols.index('month')
cols.insert(month_idx + 1, cols.pop(cols.index('day')))

# reorder DataFrame
trafficData = trafficData[cols]

3. Drop unnecessary columns:

In [8]:
# drop 'the_geom', 'the_geom_webmercator', 'cartodb_id', 'record_id', 'updated_on', 'md5', and 'date' columns
trafficData = trafficData.drop(columns=['the_geom','the_geom_webmercator','cartodb_id','record_id','updated_on','md5', 'date'])

4. Replace NaN values with 0:

In [9]:
trafficData.fillna(0, inplace=True)

5. Convert traffic volume count columns to integers:

In [10]:
# select columns with indices 11 to 34 (HOUR_00 to HOUR_23 columns)
cols_to_convert = trafficData.columns[11:35]

# Convert the selected columns to integers
trafficData[cols_to_convert] = trafficData[cols_to_convert].astype(int)

6. There are some station keys included in trafficData.csv that are not included in trafficStations.csv because of missing metadata. The total number of rows with station keys in trafficData.csv that do not match up with the station key in trafficStations.csv is 357228. As a percentage of the total length of trafficData.csv, this represents 9.35% of the total traffic data records. These rows will be removed, since the metadata cannot be left as NaN or manually filled.

In [11]:
# keep only rows where station_key exists in trafficStations
trafficData = trafficData[trafficData['station_key'].isin(trafficStations['station_key'])]

# reset the index
trafficData = trafficData.reset_index(drop=True)

7. Store cleaned DataFrame into a new .csv file:

In [12]:
trafficData.to_csv('datasets_cleaned/trafficData.csv', index=False)